In [1]:
import numpy as np
import cProfile
from imp import reload

#from info_measures import spatial as ss
import scipy.spatial as ss
import scipy as sp
#from info_measures.numpy import kraskov_stoegbauer_grassberger as ksg

In [2]:
sp.__file__

'/home/jesse/Development/scipy/scipy/__init__.py'

In [3]:
import numpy as np
import scipy.spatial as ss
x = np.random.randn(10000, 1)
xtree = ss.cKDTree(x)

In [4]:
c1 = [len(xtree.query_ball_point(point, .002, p=np.inf)) for point in x]
c2 = xtree.count_ball_point(x, .002, p=np.inf)

In [5]:
print(c1[:10])
print(c2[:10])

[18, 12, 11, 19, 5, 3, 5, 21, 7, 21]
[18 12 11 19  5  3  5 21  7 21]


# Trees

In [ ]:
x = np.random.randn(10000, 1)

In [ ]:
xtree = ss.cKDTree(x)

In [ ]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=1)

In [ ]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=-1)

In [ ]:
dvec = xtree.query(x, 3+1, p=np.inf, n_jobs=-1)[0][:, 3]

In [10]:
%timeit [len(xtree.query_ball_point(point, 50*dist - 1e-15, p=float('inf'))) for point, dist in zip(x, dvec)]

10 loops, best of 3: 91.9 ms per loop


In [ ]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, 50*dvec - 1e-15, p=float('inf'), n_jobs=-1)]

In [ ]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, 50*dvec - 1e-15, p=float('inf'), n_jobs=1)]

In [18]:
%timeit [xtree.query_ball_point(point, .02, p=np.inf) for point in x]

108 ms ± 469 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%timeit xtree.query_ball_point(x, .02, p=np.inf)

165 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit xtree.count_ball_point(x, .02, p=np.inf)

21.4 ms ± 796 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit xtree.count_ball_point(x, .02, p=np.inf, n_jobs=4)

In [ ]:
r1 = [xtree.query_ball_point(point, .002, p=np.inf) for point in x]
r2 = xtree.query_ball_point(x, .002, p=np.inf)

In [ ]:
all([np.allclose(sorted(rx1), sorted(rx2)) for rx1, rx2 in zip(r1, r2)])

In [5]:
np.allclose(c1, c2)

False

In [ ]:
%timeit xtree.query_ball_tree(xtree, 2, p=np.inf)

In [ ]:
xtree.query_ball_point(x, .2, p=np.inf)

In [ ]:
assert all([np.allclose(sorted(x), sorted(y)) for x, y in zip(xtree.query_ball_point(x, dvec - 1e-15, p=float('inf'), n_jobs=-1),
                                            [xtree.query_ball_point(point, dist - 1e-15, p=float('inf')) for point, dist in zip(x, dvec)])])

In [ ]:
nns1 = xtree.query_ball_point(x, 1., p=np.inf, n_jobs=1)

In [ ]:
nns2 = xtree.query_ball_tree(xtree, 1., p=np.inf)

In [ ]:
all([np.allclose(sorted(x), sorted(y)) for x, y in zip(nns1, nns2)])

In [ ]:
%timeit xtree.query_ball_point(x, 1., p=np.inf, n_jobs=-1)

In [ ]:
%timeit xtree.query_ball_tree(xtree, 1., p=np.inf)

# digamma

In [ ]:
%timeit ksg.avgdigamma1(xtree, x, dvec)
cProfile.run('ksg.avgdigamma1(xtree, x, dvec)')

In [ ]:
%timeit ksg.avgdigamma1_v(xtree, x, dvec, n_jobs=-1)
cProfile.run('ksg.avgdigamma1_v(xtree, x, dvec, n_jobs=-1)')

# MI

In [ ]:
x = np.random.randn(100000, 2)

In [ ]:
reload(ksg)
ksg_e = ksg.MutualInformation(x[:,[0]], x[:,[1]])
k=3
dvec = ksg_e.tree.query(ksg_e.Z, k + 1, p=float('inf'))[0][:,k]

In [ ]:
ksg_e.mutual_information()

In [ ]:
[len(xp) for xp in ksg_e.Xtree.query_ball_point(ksg_e.X, dvec)]

In [ ]:
%timeit ksg.avgdigamma1(ksg_e.Xtree, ksg_e.X, dvec)

In [ ]:
ksg_e.Xtree, ksg_e.X, dvec, n_jobs=1)

In [ ]:
%timeit ksg.avgdigamma1_v(ksg_e.Xtree, ksg_e.X, dvec, n_jobs=1)

In [ ]:
ksg_e.mutual_information(v=True)

In [ ]:
%timeit ksg_e.mutual_information(k=3)

In [ ]:
%timeit ksg_e.mutual_information(k=3, v=True)

In [ ]:
cProfile.run('ksg_e.mutual_information(n_jobs=1)')

In [ ]:
cProfile.run('ksg_e.mutual_information(n_jobs=1, v=True)')